In [1]:
import re
import cv2
import matplotlib.pyplot as plt
import scipy
import numpy as np
import matplotlib.patches as mpatches
import math
import glob
import time
import pdb
import pickle

In [2]:
def Scale_Image_And_Points(image,x,y):
    h_ratio = 227/image.shape[0]
    w_ratio = 227/image.shape[1]
    for i in range(0,len(x)):
        x[i] = int(x[i]*w_ratio)
        y[i] = int(y[i]*h_ratio)
    image = image/255
    image = cv2.resize(image, (227, 227))
    for i in range(0,len(x)):
        x[i] = x[i]/image.shape[1]
        y[i] = y[i]/image.shape[0]
    landmarks = y+x
    return image,landmarks

In [3]:
def Calculate_Pose_Estimation(image,x,y):
    im_size = image.shape
    image_points = np.array([
        (x[30], y[30]),
        (x[8], y[8]),
        (x[36], y[36]),
        (x[45],y[45]),
        (x[48], y[48]),
        (x[54],y[54])
    ], dtype = "double")

    model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner

                        ])

    #Camera Internals
    focal_length = im_size[1]
    center = (im_size[1]/2, im_size[0]/2)
    camera_matrix = np.array(
                         [[focal_length, 0, center[0]],
                         [0, focal_length, center[1]],
                         [0, 0, 1]], dtype = "double"
                         )

    #print ("Camera Matrix :\n {0}".format(camera_matrix))

    dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs)

    #print ("Rotation Vector:\n {0}".format(rotation_vector))
    #print ("Translation Vector:\n {0}".format(translation_vector))

    # Project a 3D point (0, 0, 1000.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    axis = np.float32([[500,0,0], 
                              [0,500,0], 
                              [0,0,500]])

    imgpts, jac = cv2.projectPoints(axis, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    modelpts, jac2 = cv2.projectPoints(model_points, rotation_vector, translation_vector, camera_matrix, dist_coeffs)
    rvec_matrix = cv2.Rodrigues(rotation_vector)[0]

    proj_matrix = np.hstack((rvec_matrix, translation_vector))
    eulerAngles = cv2.decomposeProjectionMatrix(proj_matrix)[6] 


    pitch, yaw, roll = [math.radians(_) for _ in eulerAngles]


    pitch = math.degrees(math.asin(math.sin(pitch)))
    roll = -math.degrees(math.asin(math.sin(roll)))
    yaw = math.degrees(math.asin(math.sin(yaw)))
    
    pose = np.array([roll,pitch,yaw])
    
    return pose

In [4]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
for i in range(500,563):
    if 0<i<10:
        prefix = '00'
        count_f = str(prefix)+str(i)
    elif 9<i<100:
        prefix = '0'
        count_f = str(prefix)+str(i)
    elif 99<i<1000:
        count_f = str(i)
    
    cap = cv2.VideoCapture(count_f + '/vid.avi')
    count = 1
    folder = count_f + '/annot/'
    dataset = np.empty((1,4))

    # Check if camera opened successfully
    if (cap.isOpened()== False): 
      print("Error opening video stream or file")

    # Read until video is completed
    while(cap.isOpened()):
      # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
        if 0<count<10:
            prefix = '00000'
            f = open(folder+str(prefix)+str(count)+'.pts',"r")
        elif 9<count<100:
            prefix = '0000'
            f = open(folder+str(prefix)+str(count)+'.pts',"r")
        elif 99<count<1000:
            prefix = '000'
            f = open(folder+str(prefix)+str(count)+'.pts',"r")
        elif 999<count<10000:
            prefix = '00'
            f = open(folder+str(prefix)+str(count)+'.pts',"r")

        if count%10 == 0:
            face = np.array([1])

            points = []
            for i,line in enumerate(f):
                if 2<i<71:
                    points.append(line)
            x = []
            y = []

            for i in range(0,len(points)):
                points[i] = re.sub(r"\n","",points[i])
                points[i] = points[i].replace("'","")
                temp_x, temp_y = points[i].split()
                x.append(int(float(temp_x)))
                y.append(int(float(temp_y)))

            pose = Calculate_Pose_Estimation(frame,x,y)
            a=max(0,y[24]-30)
            b=min(frame.shape[0],y[8]+20)
            c=max(0, x[0]-30)
            d=min(frame.shape[1], x[16]+30)
            boxA = frame[a:b, c:d]

            for i in range(0,len(x)):
                x[i] = x[i]-c
                y[i] = y[i]-a

            scaled_image, landmarks = Scale_Image_And_Points( boxA, x, y)
            data=np.array([scaled_image, face, landmarks, pose])
            data=data.reshape(1,data.shape[0])
            dataset=np.append(dataset, data, axis=0)

        # Increase the counter
        count += 1
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
          break

      # Break the loop
      else:
        dataset = np.delete(dataset,(0),axis=0)
        print("saving:",count_f)
        np.save(count_f + "/vid1"+count_f+".npy",dataset)
        del dataset
        #dataset = np.empty((1,4))
        break

    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()

Error opening video stream or file
Error opening video stream or file
Error opening video stream or file
Error opening video stream or file
Error opening video stream or file
saving: 505
saving: 506
saving: 507
saving: 508
saving: 509
saving: 510
saving: 511
Error opening video stream or file
Error opening video stream or file
saving: 514
saving: 515
saving: 516
saving: 517
saving: 518
saving: 519
saving: 520
saving: 521
saving: 522
Error opening video stream or file
saving: 524
saving: 525
saving: 526
Error opening video stream or file
saving: 528
saving: 529
saving: 530
saving: 531
Error opening video stream or file
saving: 533
Error opening video stream or file
Error opening video stream or file
Error opening video stream or file
saving: 537
saving: 538
Error opening video stream or file
saving: 540
saving: 541
Error opening video stream or file
Error opening video stream or file
Error opening video stream or file
Error opening video stream or file
saving: 546
saving: 547
saving: 54